First, let's import the required libraries and packages

In [419]:
from google.colab import drive
from pathlib import Path
import glob
import os

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

#import json to view the json data provided
import json

#basic data analysis libraries
import pandas as pd
import numpy as np
from collections import Counter

#visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px

#Text Libraries

#NLTK
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

#String
import string
from textblob import TextBlob
import re
import spacy

#sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

#Gensim
import gensim
from gensim import corpora
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rizkyario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rizkyario/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rizkyario/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading : Package '' not found in index
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rizkyario/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/rizkyario/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


Opening the JSON file using the json library

In [420]:
# Opening the JSON file
json_dir = './data/vestiairecollective'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
df = pd.concat(dfs)
print(len(df))

1440


Opening the JSON data using Pandas

In [421]:
#Using Pandas

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
reviewsDF = pd.concat(dfs)
reviewsDF.head(10)

,date,author,text,title,stars,location,url,site
0,2022-05-05,Savinee,Worst customer service! \nI sold my Dior bag w...,"Sold £1,000 bag but not paid by VC",1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
1,2022-05-05,Fanny Ann,THEY THINK THEY CAN IGNORE FOWL TREATMENT BY C...,THEY THINK THEY CAN IGNORE FOWL…,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
2,2022-05-05,David,RE: Louis Vuitton leather backpack - item: 223...,RE: REFUND,1,SA,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
3,2022-05-05,Jess Palmer,Messed up my expert badge and shadow banned i...,"Messed up my expert badge, my account and sha...",1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
4,2022-05-06,shay,I sold an authentic LV belt on this platform 2...,Loss of money and my item...,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
5,2022-05-06,Fumming,Ignorant \nUnresponsive \nTreat seller appalli...,Ignorant,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
6,2022-05-06,Jessica Greenslade,"Order placed for a pair of shoes in a UK 6.5, ...",Scammers,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
7,2022-05-06,Mouj,"I sold a Fendi bag with them, and my buyer has...","Terribe, DO NOT USE THEM",1,CA,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
8,2022-05-06,Claire,I was excited to order a vintage mulberry hand...,Counterfeit goods knowingly sold,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
9,2022-05-06,Mas,I have bought from Vestiaire a few times now. ...,BEWARE of FAKES! Authenticity checks are terri...,1,SG,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json


# **Data Cleaning**

Round 1 of cleaning: 

- Converting the text to Lowercase

- Removing text in brackets

- Removing punctuation and words

In [422]:
#Round 1 of cleaning
def clean_text(text):
    # converting to lower
    text = text.lower() 
    #remove text in square brackets
    text = re.sub('\[.*?\]','',text)
    #remove punctuation and remove words
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text = re.sub('\w*\d\w*','',text)
    return text

In [423]:
reviewsDF.title = reviewsDF.title.apply(clean_text)
reviewsDF.text = reviewsDF.text.apply(clean_text)
reviewsDF.head()

,date,author,text,title,stars,location,url,site
0,2022-05-05,Savinee,worst customer service \ni sold my dior bag wo...,sold £ bag but not paid by vc,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
1,2022-05-05,Fanny Ann,they think they can ignore fowl treatment by c...,they think they can ignore fowl…,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
2,2022-05-05,David,re louis vuitton leather backpack item \n\npu...,re refund,1,SA,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
3,2022-05-05,Jess Palmer,messed up my expert badge and shadow banned i...,messed up my expert badge my account and shadow…,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
4,2022-05-06,shay,i sold an authentic lv belt on this platform ...,loss of money and my item,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json


Round 2 of Cleaning:

- Removing additional punctuation

- Removing non-sensical text

In [424]:
#Round 2 of cleaning
def clean_text2(text):
    '''get rid of some additional punctuation and non-sesical text that was missed the first time'''
    text = re.sub('[''""...]','',text)
    text = re.sub('\n','',text)
    return text

In [425]:
reviewsDF.title = reviewsDF.title.apply(clean_text2)
reviewsDF.text = reviewsDF.text.apply(clean_text2)
reviewsDF.head()

,date,author,text,title,stars,location,url,site
0,2022-05-05,Savinee,worst customer service i sold my dior bag wort...,sold £ bag but not paid by vc,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
1,2022-05-05,Fanny Ann,they think they can ignore fowl treatment by c...,they think they can ignore fowl…,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
2,2022-05-05,David,re louis vuitton leather backpack item purcha...,re refund,1,SA,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
3,2022-05-05,Jess Palmer,messed up my expert badge and shadow banned i...,messed up my expert badge my account and shadow…,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json
4,2022-05-06,shay,i sold an authentic lv belt on this platform ...,loss of money and my item,1,GB,https://www.trustpilot.com/review/vestiairecol...,reviews_15.json


Round 3 of Cleaning:

- Punctuation

- Words containing numbers

- Stopwords

- Empty tokens

- Part of Speech Tagging

- Lemmatization

- Removing one letter words

In [426]:
#Round 3 of cleaning (postag, stopwords removal and lemmatization etc)
def get_wordnet_pos(word):
    #Map POS tag to first letter
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
    
def clean_text3(text):
    #tokenizing removing puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    #removing words containing numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    #removing stopwords
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    #removing empty tokens
    text = [t for t in text if len(t) > 0]
    #part of speech tagging
    pos_tags = pos_tag(text)
    #lemmatizing text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    #removing one letter words
    text = [t for t in text if len(t) > 1]
    #joining
    text = " ".join(text)
    return(text)

In [427]:
reviewsDF.title = reviewsDF.title.apply(clean_text3)
reviewsDF.text = reviewsDF.text.apply(clean_text3)
reviewsDF.head()

Save the Cleaned data for easy later use

In [ ]:
reviewsDF = reviewsDF.drop_duplicates(subset='text', keep="last")

#Pickling the DF for later use
reviewsDF.to_pickle("corpus.pkl")

# **Feature Engineering**

Adding new columns for
1. Length
2. Word Count
3. Polarity
4. Sentiment
5. Rating
6. Website

In [ ]:
#Let's add a column length, which indicates the length of each text
reviewsDF['length'] = reviewsDF['text'].apply(len)
#Adding a column numOfWords which indicates the number of words in the text
reviewsDF['numOfWords'] = reviewsDF['text'].apply(lambda x: len(x.split(" ")))
#Adding a Polarity column - using TextBlob
reviewsDF['polarity'] = reviewsDF['text'].map(lambda text: TextBlob(text).sentiment.polarity)
#Adding Sentiment Intensity column - using SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
reviewsDF["sentiments"] = reviewsDF["text"].apply(lambda x: sid.polarity_scores(x))
reviewsDF = pd.concat([reviewsDF.drop(['sentiments'], axis=1), reviewsDF['sentiments'].apply(pd.Series)], axis=1)
reviewsDF.head()

In [ ]:
#Adding a Sentiment column depending on the calculated compound score
reviewsDF['Sentiment'] = reviewsDF['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')
reviewsDF.Sentiment.value_counts()

In [ ]:
#Adding a column (numeric) for rating
#reviewsDF.stars.value_counts()
reviewsDF['rating'] = reviewsDF['stars']
reviewsDF.head()

In [ ]:
#Analyzing URL column
reviewsDF.url.str.split("review/")[0]
reviewsDF['Site'] = reviewsDF.url.apply(lambda x: x.split("review/")[1])
reviewsDF.head()

In [ ]:
reviewsDF.Site.value_counts()

# **EDA**

Exploratory Data Analysis

In [ ]:
#This function is used to see mathematical metrics of all the columns 
reviewsDF.describe()

In [ ]:
#To see the type of each column and check if there are any null values 
reviewsDF.date = pd.to_datetime(reviewsDF['date'])

reviewsDF.info()

In [ ]:
#To count number of samples for each rating
reviewsDF.rating.value_counts()

In [ ]:
# Review length distribution
reviewsDF['length'].hist(bins=50)

In [ ]:
#Histogram plot to see the lenght of each rating
reviewsDF.hist(column = 'length', by = 'rating')

In [ ]:
#Histogram plot to analyze number of reviews with respect to date
#As we can see, 2018 has many reviews comapared to other years
fig = px.histogram(reviewsDF, x='date', title='Reviews by date')
fig.update_xaxes(categoryorder='category descending', title='Date').update_yaxes(title='Number of reviews')
fig.show()

In [ ]:
#Histogram plot to analyze number of reviews with respect to date and ratings
#As we can see there are many one star ratings compared to all other ratings 
fig = px.histogram(reviewsDF, x='date',  title='Reviews by date', color='rating', nbins=10, log_y=True, barmode='group')
fig.update_xaxes(categoryorder='category descending', title='Date').update_yaxes(title='Number of reviews')
fig.show()

In [ ]:
#Histogram plot for polarity, and by observing we say that there are many reviwes with the polarity between 0 to 0.25
reviewsDF.polarity.hist()

In [ ]:
#Plot to analyze number of words
reviewsDF['numOfWords'].hist()

In [ ]:
#top positive reviews
reviewsDF.sort_values("pos", ascending = False)[["text", "pos", "neg"]].head(20)

In [ ]:
#Analyzing the negative data - ascending order 
reviewsDF.sort_values("neg", ascending = False)[["text", "pos", "neg"]].head(20)

In [ ]:
# reviewsDF['length'][1234]

In [ ]:
#WordCloud of text in the complete data set
wordcloud = WordCloud(max_words = 200, max_font_size = 40, scale = 3, random_state = 42).generate(str(reviewsDF['text']))
fig = plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

In [ ]:
#Percentage of each word in the wordcloud
wordcloud.words_

# **Topic Modelling**

In [ ]:
reviewData = reviewsDF['text']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reviewData = reviewData.values
document_term_matrix = tfidf_vectorizer.fit_transform(reviewData)

In [ ]:
#Let's consider the top 20 topics
n_topics = 10
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)

In [ ]:
def getCategoriesCounts(lsa_topic_matrix):
  keys = lsa_topic_matrix.argmax(axis=1).tolist()
  count_pairs = Counter(keys).items()
  categories = [pair[0] for pair in count_pairs]
  counts = [pair[1] for pair in count_pairs]
  return (keys,categories, counts)

In [ ]:
lsa_keys, lsa_categories, lsa_counts = getCategoriesCounts(lsa_topic_matrix)

In [ ]:
def get_top_n_words(n, keys, document_term_matrix, tfidf_vectorizer):
    top_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = tfidf_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [ ]:
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

We can see the top 20 topics in the whole dataset

Now let's compare the topics year by year

In [ ]:
reviewsDF.date.describe()

In [ ]:
reviewsDF.date.dt.year.value_counts() #to know the years of the available data

In [ ]:
#subset the full dataset into yearly dataframes
reviewsDF2011 = reviewsDF[reviewsDF.date.dt.year==2022]
reviewsDF2015 = reviewsDF[reviewsDF.date.dt.year==2022]
reviewsDF2016 = reviewsDF[reviewsDF.date.dt.year==2022]
reviewsDF2017 = reviewsDF[reviewsDF.date.dt.year==2022]
reviewsDF2018 = reviewsDF[reviewsDF.date.dt.year==2022]

Let's perform LSA to understand the topics for each year

In [ ]:
#2011
reviewData = reviewsDF2011['text']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reviewData = reviewData.values
document_term_matrix = tfidf_vectorizer.fit_transform(reviewData)
#Let's consider the top 2 topics
n_topics = 2
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
lsa_keys, lsa_categories, lsa_counts = getCategoriesCounts(lsa_topic_matrix)
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
top_2011 = top_n_words_lsa
for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:
#2015
reviewData = reviewsDF2015['text']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reviewData = reviewData.values
document_term_matrix = tfidf_vectorizer.fit_transform(reviewData)
#Let's consider the top 14 topics
n_topics = 10
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
lsa_keys, lsa_categories, lsa_counts = getCategoriesCounts(lsa_topic_matrix)
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
top_2015 = top_n_words_lsa
for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:
#2016
reviewData = reviewsDF2016['text']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reviewData = reviewData.values
document_term_matrix = tfidf_vectorizer.fit_transform(reviewData)
#Let's consider the top 15 topics
n_topics = 10
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
lsa_keys, lsa_categories, lsa_counts = getCategoriesCounts(lsa_topic_matrix)
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
top_2016 = top_n_words_lsa
for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:
#2017
reviewData = reviewsDF2017['text']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reviewData = reviewData.values
document_term_matrix = tfidf_vectorizer.fit_transform(reviewData)
#Let's consider the top 34 topics
n_topics = 10
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
lsa_keys, lsa_categories, lsa_counts = getCategoriesCounts(lsa_topic_matrix)
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
top_2017 = top_n_words_lsa
for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

In [ ]:
#2018
reviewData = reviewsDF2018['text']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)
reviewData = reviewData.values
document_term_matrix = tfidf_vectorizer.fit_transform(reviewData)
#Let's consider the top 12 topics
n_topics = 12
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
lsa_keys, lsa_categories, lsa_counts = getCategoriesCounts(lsa_topic_matrix)
top_n_words_lsa = get_top_n_words(3, lsa_keys, document_term_matrix, tfidf_vectorizer)
top_2018 = top_n_words_lsa
for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i+1), top_n_words_lsa[i])

Visualizing the word cloud for each year to understand the most common words

In [ ]:
#2011
wordcloud2011 = WordCloud(max_words = 200, max_font_size = 40, scale = 3, random_state = 42).generate(str(reviewsDF2011['text']))
fig = plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud2011)
plt.show()

In [ ]:
#2015
wordcloud2015 = WordCloud(max_words = 200, max_font_size = 40, scale = 3, random_state = 42).generate(str(reviewsDF2015['text']))
fig = plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud2015)
plt.show()

In [ ]:
#2016
wordcloud2016 = WordCloud(max_words = 200, max_font_size = 40, scale = 3, random_state = 42).generate(str(reviewsDF2016['text']))
fig = plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud2016)
plt.show()

In [ ]:
#2017
wordcloud2017 = WordCloud(max_words = 200, max_font_size = 40, scale = 3, random_state = 42).generate(str(reviewsDF2017['text']))
fig = plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud2017)
plt.show()

In [ ]:
#2018
wordcloud2018 = WordCloud(max_words = 200, max_font_size = 40, scale = 3, random_state = 42).generate(str(reviewsDF2018['text']))
fig = plt.figure(1, figsize = (20, 20))
plt.axis('off')
plt.imshow(wordcloud2018)
plt.show()

In [ ]:
#Let's create a df with the top topics for all the years
l1,l5,l6,l7,l8=len(top_2011),len(top_2015),len(top_2016),len(top_2017),len(top_2018)
max_len = max(l1,l5,l6,l7,l8)

if not max_len == l1:
  top_2011.extend(['-']*(max_len-l1))
if not max_len == l5:
  top_2015.extend(['-']*(max_len-l5))
if not max_len == l6:
  top_2016.extend(['-']*(max_len-l6))
if not max_len == l7:
  top_2017.extend(['-']*(max_len-l7))
if not max_len == l8:
  top_2018.extend(['-']*(max_len-l8))

topicsDF = pd.DataFrame({'2011':top_2011,'2015':top_2015,'2016':top_2016,'2017':top_2017,'2018':top_2018})
topicsDF

Even though we have the top topics for each year, it is difficult to infer how the trend has changed over the years!  

In [ ]:
reviewsDF

As the number of reviews in 2011 is very less, lets plot the polarity for the remaining years in terms of time

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2,figsize=(15,10))
#reviewsDF2011.plot.line(x='date',y='polarity',ax=axes[0,0])
reviewsDF2015.plot.line(x='date',y='polarity',ax=axes[0,0],title='2015')
reviewsDF2016.plot.line(x='date',y='polarity',ax=axes[0,1],title='2016')
reviewsDF2017.plot.line(x='date',y='polarity',ax=axes[1,0],title='2017')
reviewsDF2018.plot.line(x='date',y='polarity',ax=axes[1,1],title='2018')

In [ ]:
#Let's create a DF with the total number of positive and negative reviews detected for each year
sentimentDF = pd.DataFrame(0,range(5),columns=['Year','pos','neg'])
sentimentDF

In [ ]:
sentimentDF.loc[0] = [2011,reviewsDF2011.Sentiment.value_counts()['pos'],reviewsDF2011.Sentiment.value_counts()['neg']]
sentimentDF.loc[1] = [2015,reviewsDF2015.Sentiment.value_counts()['pos'],reviewsDF2015.Sentiment.value_counts()['neg']]
sentimentDF.loc[2] = [2016,reviewsDF2016.Sentiment.value_counts()['pos'],reviewsDF2016.Sentiment.value_counts()['neg']]
sentimentDF.loc[3] = [2017,reviewsDF2017.Sentiment.value_counts()['pos'],reviewsDF2017.Sentiment.value_counts()['neg']]
sentimentDF.loc[4] = [2018,reviewsDF2018.Sentiment.value_counts()['pos'],reviewsDF2018.Sentiment.value_counts()['neg']]

In [ ]:
sentimentDF

In [ ]:
#Plot of positive vs negative reviews over the years
sentimentDF.plot.line(x='Year')

# **LDA & Topic trend**

Now, even though the LSA provided the topics, let's analyze the topics using LDA in order to analyze the trend and to better visualize the topics 

In [ ]:
df = reviewsDF
df

In [ ]:
warnings.filterwarnings('ignore')

reviews = list(df['text'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
reviews = pd.Series(reviews).apply(lambda x: x.split())
r1 = []
for wrd in reviews:
  doc = nlp(" ".join(wrd))
  r1.append([token.lemma_ for token in doc if token.pos_ in ['NOUN','ADJ']])

reviewsR1 = r1

In [ ]:
#LDA
dictionary = corpora.Dictionary(reviewsR1)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviewsR1]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel
# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=42,chunksize=1000, passes=50)

In [ ]:
#The top 20 topics that the LDA model has learned are
lda_model.print_topics()

In [ ]:
#Just to check the correctness
lda_model.get_document_topics(doc_term_matrix[1])

Using the **pyLDAvis** let's visualize the topics and relations

In [ ]:
#Visualizing using the pyLDAvis inbuilt viz
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

In [ ]:
#Let's add a new column in the main df to get the topic number for each review
doc_lda = lda_model[doc_term_matrix]
max_topics = [max(sent, key=lambda x: x[1])[0] for sent in doc_lda]

In [ ]:
dftopic = df
dftopic['topic'] = max_topics

In [ ]:
dftopic

In [ ]:
#the topics for the whole df
dftopic.topic.value_counts()

In [ ]:
#Topic numbers for the year 2011
topic2011 = dftopic[dftopic.date.dt.year==2011]['topic'].value_counts().index.tolist()

In [ ]:
#Topic numbers for the year 2015
topic2015 = dftopic[dftopic.date.dt.year==2015]['topic'].value_counts().index.tolist()

In [ ]:
#Topic numbers for the year 2016
topic2016 = dftopic[dftopic.date.dt.year==2016]['topic'].value_counts().index.tolist()

In [ ]:
#Topic numbers for the year 2017
topic2017 = dftopic[dftopic.date.dt.year==2017]['topic'].value_counts().index.tolist()

In [ ]:
#Topic numbers for the year 2018
topic2018 = dftopic[dftopic.date.dt.year==2018]['topic'].value_counts().index.tolist()

The topics learned for each Year

In [ ]:
print("2011 - ",topic2011,"\n2015 - ",topic2015,"\n2016 - ",topic2016,"\n2017 - ",topic2017,"\n2018 - ",topic2018)

The list of all the topics learned with the LDA model

In [ ]:
for i in range(len(lda_model.print_topics())):
  print("Topic:%i" %i ," - ", lda_model.print_topics()[i][1], '\n')

We can see from above that:

In 2011: There are only 2 topics - Topic #15 and Topic#0

In [ ]:
lda_model.print_topics()[0][1]

In [ ]:
list(set(topic2011).intersection(topic2015))

From 2011 to 2015 there are two common topics

In [ ]:
list(set(topic2015).intersection(topic2016))

We can see that from 2015 to 2016 Topic #0 and #15 have been continued, and many more topics have been added

In [ ]:
list(set(topic2016).intersection(topic2017))

From 2016 to 2017, most of the topics are still common and some more have been added

In [ ]:
list(set(topic2017).intersection(topic2018))

Again from 2017 to 2018, most of the topics have still been common 

In [ ]:
for i in range(20):
  print("Topic:%i" %i )
  print(df[df.topic == i]['rating'].value_counts().index[0])

we can see that almost all the topics have the highest number of negative reviews except Topic #6 and #3

From the above learned topics, let's consider one of the most common topic #10 that has been talked about since 2015, which indicates it's one of the most important things people are talking about 

In [ ]:
#Let's see some reviews with the topic #10
list(df[df.topic == 10]['text'][:20])

In [ ]:
df[df.topic == 10]['rating'].value_counts()

We can see from above that most of the reviews are negative, and almost more thatn 75% of the reviews have a 1-star rating. So, we can confirm that this is one of the most important topic people have been talking about over the years.

Similarly we can analyze the rest of the topics depending on the requirement, and how the service wants to be improved, for example if they want to improve the Application, they can see the topic most related to "app" and then do the same analysis as above. 